In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils

# Define directories
image_directory = '../data/deigo/annotate-frames'  # Original Images
depth_map_directory = '../data/deigo/depth-maps/stingray-vid/'  # Depth Map Images
json_file = '../data/deigo/diego-pipe-annotations/annotations/instances_default.json'  # JSON file

# Path to save results
results_csv_path = '../data/deigo/results_stingray_vid.csv'

# Images and their corresponding annotations
image_names = [
    'stingray_frame_000001.png',
    'stingray_frame_000002.png',
    'stingray_frame_000003.png'
]

# Load the COCO annotations
coco = COCO(json_file)

# Create a mapping from category_id to category_name
cats = coco.loadCats(coco.getCatIds())
category_mapping = {cat['id']: cat['name'] for cat in cats}
print("Category mapping:")
print(category_mapping)

def load_and_normalize_depth_map(depth_map_path):
    # Load the depth map image
    depth_map = cv2.imread(depth_map_path, cv2.IMREAD_UNCHANGED)
    if depth_map is None:
        print(f"Failed to load depth map at {depth_map_path}.")
        return None

    # Normalize the depth map based on its data type
    if depth_map.dtype == np.uint8:
        # 8-bit depth map
        depth_map_normalized = depth_map / 255.0
    elif depth_map.dtype == np.uint16:
        # 16-bit depth map
        depth_map_normalized = depth_map / 65535.0
    else:
        # If depth map is already in float format between 0 and 1
        depth_map_normalized = depth_map

    return depth_map_normalized

def create_mask_from_annotation(ann, img_height, img_width):
    segmentation = ann['segmentation']

    if isinstance(segmentation, list):  # Polygon format
        mask = np.zeros((img_height, img_width), dtype=np.uint8)
        # Convert polygons to a mask
        for poly in segmentation:
            poly = np.array(poly).reshape((-1, 2))  # Reshape to (N, 2)
            cv2.fillPoly(mask, [np.int32(poly)], 1)  # Fill the polygon with 1's

    elif isinstance(segmentation, dict) and 'counts' in segmentation:  # RLE format
        rle = segmentation
        if isinstance(rle['counts'], list):  # Uncompressed RLE
            rle = maskUtils.frPyObjects([rle], img_height, img_width)
        mask = maskUtils.decode(rle)  # Decode RLE to binary mask

    else:
        mask = None  # Segmentation format not recognized

    return mask

# Load annotations for each image
annotations_dict = {}
image_sizes = {}

# Load all images and create a mapping from file_name to image ID
all_images = coco.loadImgs(coco.getImgIds())
file_name_to_id = {img['file_name']: img['id'] for img in all_images}

for img_name_with_ext in image_names:
    img_id = file_name_to_id.get(img_name_with_ext)
    if img_id is None:
        print(f"Image {img_name_with_ext} not found in annotations.")
        continue
    img_info = coco.loadImgs(img_id)[0]
    img_height = img_info['height']
    img_width = img_info['width']
    image_sizes[img_name_with_ext] = (img_width, img_height)
    ann_ids = coco.getAnnIds(imgIds=[img_id])
    anns = coco.loadAnns(ann_ids)
    annotations_dict[img_name_with_ext] = anns

print(f"Loaded annotations for images: {image_names}")

# Get and sort depth maps
depth_map_files = [f for f in os.listdir(depth_map_directory) if f.endswith('_pred04.png')]
depth_map_files.sort()  # Sort files to ensure correct order

num_depth_maps = len(depth_map_files)
print(f"Total number of depth maps: {num_depth_maps}")

# Calculate indices for dividing into thirds
third = num_depth_maps // 3
remainder = num_depth_maps % 3

indices = [
    (0, third),
    (third, 2 * third),
    (2 * third, 3 * third + remainder)
]

# Map each depth map to the corresponding image's annotations
results = []

for idx, (start_idx, end_idx) in enumerate(indices):
    img_name_with_ext = image_names[idx]
    if img_name_with_ext not in annotations_dict:
        print(f"No annotations found for image {img_name_with_ext}. Skipping this set of depth maps.")
        continue
    anns = annotations_dict[img_name_with_ext]
    img_width, img_height = image_sizes[img_name_with_ext]

    depth_maps_subset = depth_map_files[start_idx:end_idx]
    print(f"\nProcessing depth maps {start_idx} to {end_idx - 1} using annotations from {img_name_with_ext}")

    for depth_map_file in depth_maps_subset:
        depth_map_path = os.path.join(depth_map_directory, depth_map_file)
        depth_map = load_and_normalize_depth_map(depth_map_path)
        if depth_map is None:
            continue  # Skip if depth map couldn't be loaded

        depth_map_file_name = os.path.basename(depth_map_file)
        print(f"Processing depth map: {depth_map_file_name}")

        # Process each annotation
        for ann in anns:
            annotation_id = ann['id']
            category_id = ann['category_id']
            category_name = category_mapping.get(category_id, "Unknown")
            print(f"Processing annotation ID {annotation_id}, category ID {category_id}, category name {category_name}")

            # Create mask for the annotation
            mask = create_mask_from_annotation(ann, img_height, img_width)
            if mask is None:
                print(f"Failed to create mask for annotation {annotation_id}")
                continue  # Skip if mask couldn't be created

            # Resize depth map if necessary
            if depth_map.shape != mask.shape:
                depth_map_resized = cv2.resize(depth_map, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
            else:
                depth_map_resized = depth_map

            # Apply the mask to the depth map to extract depth values
            depth_values = depth_map_resized[mask == 1]
            mean_depth = depth_values.mean() if depth_values.size > 0 else 0
            normalized_depth = mean_depth  # Depth map is already normalized between 0 and 1

            # Convert normalized depth to metric depth
            zmin = 0.4  # Minimum depth in meters (adjust if necessary)
            zmax = 20.0  # Maximum depth in meters (adjust if necessary)
            value_metric = (normalized_depth * (zmax - zmin)) + zmin

            print(f"Annotation ID {annotation_id}: mean depth {mean_depth}, normalized depth {normalized_depth}, value_metric {value_metric}")

            # Append the result to the list
            results.append({
                'depth_map_file': depth_map_file_name,
                'annotation_image': img_name_with_ext,
                'annotation_id': annotation_id,
                'category_id': category_id,
                'category_name': category_name,
                'normalized_depth': normalized_depth,
                'z-min': zmin,
                'z-max': zmax,
                'value_metric': value_metric
            })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the DataFrame
print("\nResults DataFrame:")
print(results_df.head())

# Save the results to a CSV file
results_df.to_csv(results_csv_path, index=False)
print(f"\nResults saved to {results_csv_path}")


loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../data/deigo/diego-pipe-annotations/annotations/instances_default.json'